In [1]:
import numpy as np
import cv2
from tensorflow import keras
import pandas as pd

In [2]:
def denoise_image(img):
    #img is a numpy array that was earlier processed by median filter
    model=keras.model.load_model("no_skip_conn_encoder.h5")
    ans=model.predict(np.expand_dims(img,axis=0))
    return ans[0]
def median_filter(data, filter_size):
    temp = []
    indexer = filter_size // 2
    data_final = []
    data_final = np.zeros((len(data),len(data[0])))
    for i in range(len(data)):

        for j in range(len(data[0])):

            for z in range(filter_size):
                if i + z - indexer < 0 or i + z - indexer > len(data) - 1:
                    for c in range(filter_size):
                        temp.append(0)
                else:
                    if j + z - indexer < 0 or j + indexer > len(data[0]) - 1:
                        temp.append(0)
                    else:
                        for k in range(filter_size):
                            temp.append(data[i + z - indexer][j + k - indexer])

            temp.sort()
            data_final[i][j] = temp[len(temp) // 2]
            temp = []
    return data_final
def salt_pepper_denoise(img,filter_size):
    for i in range(3):
        img[:,:,i]=median_filter(img[:,:,i],2)
    return img
# the function salt_pepper_denoise is the function that will be called.

In [3]:
def sharpen_image(img):
    sharpkernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    image_sharp = cv2.filter2D(src=img, ddepth=-1, kernel=sharpkernel)
    return image_sharp

In [4]:
# Need to add tesseract to the pipeline after the sharp filter has been applied so as to produce the final text
# after the final text has been produced it will be sent to NLP block for correcting the wrong words.
# The function below must add the tesseract part

def pipeline_composed(path):
    img=cv2.imread(path)
    img=cv2.resize(img,(600,600))
    img = salt_pepper_denoise(img,3)
    img=img/255.0
    img=denoise_img(img)
    return img
# Need to add Tesseract and NLP block.